# <font style="color:rgb(50,120,229)">Grabcut for Background Removal </font>
In this notebook, we will describe the working of Grabcut in OpenCV with the help of an example.

GrabCut is an interactive segmentation method. It is used to separate an image into a background and a foreground. Let’s see a typical way of using GrabCut interactively. 

# <font style="color:rgb(50,120,229)">GrabCut in OpenCV</font>

GrabCut is implemented in OpenCV using a function named [**`grabCut`**](https://docs.opencv.org/4.1.0/d7/d1b/group__imgproc__misc.html#ga909c1dda50efcbeaa3ce126be862b37f).Let us look at the usage. 

### <font style = "color:rgb(8,133,37)">Function Syntax</font>

```python
mask, bgdModel, fgdModel	=	cv.grabCut(	img, mask, rect, bgdModel, fgdModel, iterCount[, mode]	)
```

**Parameters**
- **`img`**:	Input 8-bit 3-channel image.
- **`mask`**:	Input/output 8-bit single-channel mask. The mask is initialized by the function when mode is set to GC_INIT_WITH_RECT. Its elements may have one of the GrabCutClasses.
- **`rect`**:	ROI containing a segmented object. The pixels outside of the ROI are marked as "obvious background". The parameter is only used when mode==GC_INIT_WITH_RECT .
- **`bgdModel`**:	Temporary array for the background model. Do not modify it while you are processing the same image.
- **`fgdModel`**:	Temporary arrays for the foreground model. Do not modify it while you are processing the same image.
- **`iterCount`**:	Number of iterations the algorithm should make before returning the result. Note that the result can be refined with further calls with mode==GC_INIT_WITH_MASK or mode==GC_EVAL .
- **`mode`**:	Operation mode that could be one of the GrabCutModes

The code snippet shows the usage of the code. 

In [ ]:
"""
===============================================================================
Interactive Image Segmentation using GrabCut algorithm.


USAGE:
    python grabcut.py <filename>
"""

## <font style="color:rgb(50,120,229)">Functionality </font>
Since it is an interactive application, we are using the mouse and keyboard to get inputs from the user and update the results.

When you run the code, Two windows will show up,  one for input and one for output. Then follow these steps
1. In input window, draw a rectangle around the object using the mouse. Then press 'n' to segment the object (once or a few times)
1. For making further changes to the output,  you can press any of the keys below and scribble on the areas you want. Then again press 'n' for updating the output.

    ### <font style="color:rgb(8,133,37)">Select the mode </font>
    - Key '0' - To select areas of sure background
    - Key '1' - To select areas of sure foreground
    - Key '2' - To select areas of probable background
    - Key '3' - To select areas of probable foreground

    ### <font style="color:rgb(8,133,37)">Update </font>
    - Key 'n' - To update the segmentation
    - Key 'r' - To reset the setup
    - Key 's' - To save the results
    - Key 'Esc'- To Exit


In [ ]:
import numpy as np
import cv2
import sys

#### <font style = "color:rgb(8,133,37)">Variables defining the different colors used </font>

In [ ]:
# Convention of defining color in opencv is BGR
LIGHT_GREEN = [128, 255, 128]        # rectangle color
LIGHT_RED = [128, 128, 255]         # PR BG
BLUE = [255, 0, 0]        # rectangle color
RED = [0, 0, 255]         # PR BG
GREEN = [0, 255, 0]       # PR FG
BLACK = [0, 0, 0]         # sure BG
WHITE = [255, 255, 255]   # sure FG

#### <font style = "color:rgb(8,133,37)">Variables defining the different modes</font>

In [ ]:
# Creating Dictionary
DRAW_BG = {'color' : RED,  'val' : 0}
DRAW_FG = {'color' : GREEN,  'val' : 1}
DRAW_PR_FG = {'color' : LIGHT_GREEN,  'val' : 3}
DRAW_PR_BG = {'color' : LIGHT_RED,  'val' : 2}

#### <font style = "color:rgb(8,133,37)">Variables defining the different flags and global variables</font>

In [ ]:
# Setting up flags
rect = (0, 0, 1, 1)
drawing = False         # flag for drawing curves
rectangle = False       # flag for drawing rect
rect_over = False       # flag to check if rect drawn
rect_or_mask = 100      # flag for selecting rect or mask mode
value = DRAW_FG         # drawing initialized to FG
thickness = 3           # brush thickness
rect_not_done = True

### <font style="color:rgb(8,133,37)">Mouse Callback Function </font>
We use the mouse for drawing the initial rectangle and then scribbling in later iterations. 

1. The first if-else loop monitors the mouse movements and draws the rectangle. It also keeps track of the points pressed by the user and passes the coordinates to the grabcut algorithm later.
1. The second if-else loop keeps track of the mouse movements and whenever the mouse button is pressed, it updates the the `mask`.


In [ ]:
# Application Function on mouse
def onmouse(event, x, y, flags, param):
    global img, img2, drawing, value, mask, rectangle, rect, rect_or_mask, ix, iy, rect_over,  rect_not_done

    # Draw Rectangle
    if (event == cv2.EVENT_LBUTTONDOWN) and rect_not_done:
        rectangle = True
        ix, iy = x, y

    elif event == cv2.EVENT_MOUSEMOVE:
        if rectangle == True:
            img = img2.copy()
            cv2.rectangle(img, (ix, iy), (x, y), BLUE, 2)
            rect = (min(ix, x), min(iy, y), abs(ix-x), abs(iy-y))
            rect_or_mask = 0

    elif (event == cv2.EVENT_LBUTTONUP) and rect_not_done:
        rectangle = False
        rect_not_done = False
        rect_over = True
        cv2.rectangle(img, (ix, iy), (x, y), BLUE, 2)
        rect = (min(ix, x), min(iy, y), abs(ix-x), abs(iy-y))
        rect_or_mask = 0
        print(" Now press the key 'n' a few times until no further change \n")

    # Draw touchup curves
    if event == cv2.EVENT_LBUTTONDOWN:
        if rect_over == False:
            print("first draw rectangle \n")
        else:
            drawing = True
            cv2.circle(img, (x, y), thickness, value['color'], -1)
            cv2.circle(mask, (x, y), thickness, value['val'], -1)

    elif event == cv2.EVENT_MOUSEMOVE:
        if drawing == True:
            cv2.circle(img, (x, y), thickness, value['color'], -1)
            cv2.circle(mask, (x, y), thickness, value['val'], -1)

    elif event == cv2.EVENT_LBUTTONUP:
        if drawing == True:
            drawing = False
            cv2.circle(img, (x, y), thickness, value['color'], -1)
            cv2.circle(mask, (x, y), thickness, value['val'], -1)

### <font style="color:rgb(8,133,37)">The main function </font>
We read an image for performing grabcut background segmentation. Then we create the windows for input and output images and setup the mouse callback functions so that the output window is updated depending on the user input.

In the above function, we discussed the mouse events. In this function, we will see how the keyboard events are handled.

1. We first check for the mode to be used - It can be 
    - 0 : Background
    - 1 : Foreground
    - 2 : Probable Background
    - 3 : Probable Background
2. Next we check for keys `s` which saves the output image and key `r` which resets the `mask` and `output`.
3. Next we check for the key `n` which means grabcut needs to be applied and the `output` need to be updated.
4. Finally we merge the pixels which are marked as definitely foreground and probably foreground and update the `mask` and the `output`.

In [ ]:
if __name__ == '__main__':

    # print documentation
    print(__doc__)

    # Loading images if image is given bu command line
    if len(sys.argv) == 2:
        filename = sys.argv[1] # Using file for image
        print("Loading Image \n")
    else:
        print("No input image given,  so loading default image,  ../../data/images/hillary_clinton.jpg \n")
        print("Correct Usage: python grabcut.py <filename> \n")
        filename = '../data/images/hillary_clinton.jpg'

    img = cv2.imread(filename)
    img2 = img.copy()                               # a copy of original image
    mask = np.zeros(img.shape[:2], dtype = np.uint8) # mask initialized to PR_BG
    output = np.zeros(img.shape, np.uint8)           # output image to be shown

    # input and output windows
    cv2.namedWindow('output')
    cv2.namedWindow('input')
    cv2.setMouseCallback('input', onmouse)
    cv2.moveWindow('input', img.shape[1]+10, 90)

    print(" Instructions: \n")
    print(" Draw a rectangle around the object using right mouse button \n")

    while(1):

        cv2.imshow('output', output)
        cv2.imshow('input', img)
        k = cv2.waitKey(1)

        # key bindings
        if k == 27:                                  # esc to exit
            break
        elif k == ord('0'):                          # BG drawing
            print(" Using Red color,  >mark background regions with left mouse button \n")
            value = DRAW_BG
        elif k == ord('1'):                          # FG drawing
            print(" Using Green color, >mark foreground regions with left mouse button \n")
            value = DRAW_FG
        elif k == ord('2'):                          # PR_BG drawing
            print(" Using Light Red color, >mark probable Background regions with left mouse button \n")
            value = DRAW_PR_BG
        elif k == ord('3'):                          # PR_FG drawing
            print(" Using Light Green color, >mark probable foreground regions with left mouse button \n")
            value = DRAW_PR_FG

        elif k == ord('s'):                          # save image
            bar = np.zeros((img.shape[0], 5, 3), np.uint8)
            res = np.hstack((img2, bar, img, bar, output))
            cv2.imwrite('grabcut_output.png', res)
            print(" Result saved as image \n")

        elif k == ord('r'):                          # reset everything
            print("resetting \n")
            rect = (0, 0, 1, 1)
            drawing = False
            rectangle = False
            rect_or_mask = 100
            rect_over = False
            rect_not_done = True
            value = DRAW_FG
            img = img2.copy()
            mask = np.zeros(img.shape[:2], dtype = np.uint8) # mask initialized to PR_BG
            output = np.zeros(img.shape, np.uint8)    # output image to be shown
            print(__doc__)

        elif k == ord('n'):                         # segment the image
            print(""" For finer touchups,  mark foreground and background after pressing keys 0-3
            and again press 'n' \n""")

            if (rect_or_mask == 0):                 # grabcut with rect
                bgdmodel = np.zeros((1, 65), np.float64)
                fgdmodel = np.zeros((1, 65), np.float64)
                cv2.grabCut(img2, mask, rect, bgdmodel, fgdmodel, 1, cv2.GC_INIT_WITH_RECT)
                rect_or_mask = 1

            elif rect_or_mask == 1:                 # grabcut with mask
                bgdmodel = np.zeros((1, 65), np.float64)
                fgdmodel = np.zeros((1, 65), np.float64)
                cv2.grabCut(img2, mask, rect, bgdmodel, fgdmodel, 1, cv2.GC_INIT_WITH_MASK)

        # Final mask is the union of definitely foreground and probably foreground
        # mask such that all 1-pixels (cv2.GC_FGD) and 3-pixels (cv2.GC_PR_FGD) are put to 1 (ie foreground) and
        # all rest are put to 0(ie background pixels)
        mask2 = np.where((mask==1) + (mask==3), 255, 0).astype('uint8')

        # Copy the region to output
        output = cv2.bitwise_and(img2, img2, mask=mask2)

    cv2.destroyAllWindows()

Do try out the code with your own inputs. This application can be used to create image annotation tools for segmentation to create large datasets.